In [133]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [186]:
%%time
import pandas as pd
from sklearn import metrics,cluster
import numpy as np
import random

#### Reading the feature and calculating the pairwise distance between all the points

df = pd.read_csv('gh_lyso_PAAC.txt', sep='\t')# input file which contains features, header=None if no column title
x=df.iloc[:,1:].values

# distance=metrics.pairwise.pairwise_distances(x)

def cal_dist(b,c):
    temp=np.zeros([len(x),len(b)])
    for i1 in range(len(x)):
        for j1 in b:
            temp_dis=np.sqrt(sum([(xx-yy)**2 for xx,yy in zip(x[i1],c[j1])]))
            temp[i1][j1]=temp_dis
    return temp

#### collecting intra and inter distance for clusters


class clust:
    def __init__(self,ref,comp):
        self.ref=ref
        self.comp=comp
        if self.ref==self.comp:
#             print(f'The intra cluster distance within cluster {self.ref}')
            self.c=1
        else:
#             print(f'The inter cluster distance between cluster {self.ref} and {self.comp}')
            self.c=0
        
    def all_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            for y1 in range((x1+1)*self.c,len(labels)):
                if labels[x1]==self.ref and labels[y1]==self.comp:
#                     print(f'{x1}..{y1}',distance[x1][y1])
                    al_data.append(distance[x1][y1])
        self.data=al_data
        self.avg=sum(al_data)/len(al_data)
        return self

    def centroid_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            if labels[x1]==self.ref:
                al_data.append(cent_dist[x1][self.comp])
        self.data=al_data
        self.avg=sum(al_data)/len(al_data)
        return self               

def un_learn(n):
    kmeans = cluster.KMeans(n_clusters=n).fit(x) # Number of clusters
    lab = kmeans.labels_
    centroid=kmeans.cluster_centers_
    cent_lab=kmeans.predict(centroid)
    c_d=cal_dist(cent_lab,centroid)
    return lab,centroid,c_d

labels,cent,cent_dist=un_learn(3)
a=clust(0,0).centroid_dist() # cluster compared to centroid

Wall time: 49.8 ms


In [187]:
%%time
from collections import Counter
from itertools import combinations
import numpy as np

# a=clust(0,1).all_dist()
# b=clust(0,0).all_dist()
##########################################################################
nn=3 # number of cluster
ni=100 # number of iterations
##########################################################################
lis=[s for s in range(nn)]
grp=list(combinations(lis,2))
grp.extend([(i,i) for i in lis])

grp_taken=grp

min_dis,max_dis,avg_dis={},{},{}
for each in grp_taken:
    min_dis[each],max_dis[each],avg_dis[each]=[],[],[]
    
for each_iter in range(ni):
#     print(f'Iteration:{each_iter+1}')
#     labels,no_lab=un_learn(nn)
    labels,cent,cent_dist=un_learn(3)
    for j in grp_taken:
#         c=clust(j[0],j[1]).all_dist()
        c=clust(j[0],j[0]).centroid_dist() # cluster compared to centroid
        min_dis[j].append(min(c.data))
        max_dis[j].append(max(c.data))
        avg_dis[j].append(c.avg)

Wall time: 4.6 s


In [188]:
main_list=[]
for each_iter in range(ni):
#     print(f'Iteration:{each_iter+1}')
    emp_array_avg=np.zeros((len(cent),len(cent)))
    emp_array_m=np.zeros((len(cent),len(cent)))
    sub_list=[]
    for k in grp_taken:
        if k[0]==k[1]:
            emp_array_m[k[0]][k[1]]=max_dis[k][each_iter]
            sub_list.append(max_dis[k][each_iter])
    main_list.append(min(sub_list))
    
#         print(min(sub_list))
#         else:
#             emp_array_m[k[1]][k[0]]=min_dis[k][each_iter]
#             emp_array_m[k[0]][k[1]]=min_dis[k][each_iter]
# df1=pd.DataFrame(emp_array_m)
# df1

In [113]:
out=open('min_max_intra_cent_km.txt','w')
for line in main_list:
    out.write(f'{line}\n')
out.close()

In [181]:
min(main_list)

0.017790729803026702

In [189]:
Counter(main_list)# lysozyme

Counter({0.024427904756936757: 100})

In [182]:
Counter(main_list)# hemoglobin

Counter({0.017790729803026702: 92,
         0.018657292036083805: 6,
         0.018683315608651628: 1,
         0.020138663431649415: 1})

In [175]:
Counter(main_list)# cellulases

Counter({0.02334451044011748: 9,
         0.029516337953961497: 61,
         0.02424619302646035: 21,
         0.023761131912604774: 5,
         0.029197061000493584: 1,
         0.02191390861299122: 1,
         0.023526093971853176: 1,
         0.020343681474981584: 1})